---

##  Resumen Ejecutivo del Pipeline

### Transformación Completa

| Métrica | Antes | Después | Cambio |
|---------|-------|---------|--------|
| **Registros totales** | 5,955,075 | 2,082,071 | Filtrado HE 2017-2024 |
| **Duplicados** | 9,165 (0.15%) | 0 | ✓ Eliminados |
| **Archivos fuente** | 11 CSVs (2014-2024) | 1 dataset unificado | ✓ Consolidado |
| **Columnas** | 19-21 (inconsistente) | 26 (estandarizadas) | ✓ Normalizado |
| **Formato de fechas** | Mixto | ISO 8601 | ✓ Estandarizado |
| **Nombres de países** | Múltiples variantes | ISO2 + nombre canónico | ✓ Unificado |

### Calidad de Datos

 **100% de países normalizados** por código ISO2
 **0 duplicados exactos**
 **621,084 edades outliers** convertidas a NA (outliers imposibles <10 o >65)
 **16,069 duraciones = 0** convertidas a NA
 **Tokens de nulos** unificados ("unknown", "n/a", "???", etc. → NA)

### Dataset Final para Power BI

- **Enfoque**: Educación Superior (ISCED 6-8)
- **Período**: 2017-2024 (años de inicio de movilidad)
- **Variables**: 26 columnas analíticas
- **Formatos**: Parquet (comprimido) + CSV (universal)

---

Para detalles técnicos del proceso de limpieza, ver [`PIPELINE.md`](PIPELINE.md)

In [1]:
#Importamos las librerías necesarias para el proceso
import numpy as np
import pandas as pd
import os
import re
import unicodedata

In [ ]:
# - Compruebo si los CSV anuales (2014–2024) tienen la misma estructura de columnas.
# - Detecto diferencias entre años antes de hacer la carga completa.

# - Normalizo nombres de columnas (minúsculas + guiones bajos) para comparar de forma consistente.
folder_path = "bases_datos_erasmus"
years = range(2014, 2025)  # incluye 2024


def normalize_columns(columns):
    """
    Normaliza nombres de columnas para comparar entre años.
    En este bloque hago una normalización básica (minúsculas y guiones bajos).
    """
    normalized = []
    for col in columns:
        col = col.strip().lower()
        col = re.sub(r"\s+", " ", col)      
        col = col.replace(" ", "_")         
        normalized.append(col)
    return normalized


def read_header_with_fallback(file_path, sep=";"):
    """
    Lee SOLO la cabecera del CSV (nrows=0) probando varios encodings.
    Devuelve (df_header, encoding_usado).
    """
    encodings_to_try = ["utf-8", "utf-8-sig", "latin1"]
    last_error = None

    for enc in encodings_to_try:
        try:
            df = pd.read_csv(file_path, sep=sep, nrows=0, encoding=enc)
            return df, enc
        except Exception as e:
            last_error = e

    raise last_error


column_sets = {}   # columnas (normalizadas) por año
errors = {}        # errores de lectura por año

print("Verificando estructura de archivos...")

for year in years:
    file_name = f"Erasmus-KA1-Mobility-Data-{year}.csv"
    file_path = os.path.join(folder_path, file_name)

    try:
        df_header, used_enc = read_header_with_fallback(file_path, sep=";")
        column_sets[year] = normalize_columns(df_header.columns)
        print(f"  {year}: {len(df_header.columns)} columnas (encoding={used_enc})")

    except Exception as e:
        errors[year] = str(e)
        print(f"  {year}: Error leyendo {file_name}: {e}")


print("\nComparando columnas entre años...")

if not column_sets:
    print("No se pudo leer ningún archivo. Revisa rutas, nombres y separador.")
else:
    # Tomo como referencia el primer año que se leyó correctamente
    base_year = min(column_sets.keys())
    base_cols = set(column_sets[base_year])

    for year in sorted(column_sets.keys()):
        if year == base_year:
            continue

        current_cols = set(column_sets[year])

        if current_cols != base_cols:
            print(f"\n  Diferencias detectadas en {year} (base={base_year}):")
            print("    - Faltan:", base_cols - current_cols)
            print("    - Nuevas:", current_cols - base_cols)
        else:
            print(f"  {year} tiene las mismas columnas que {base_year}.")


print("\nResumen:")
print("  Años OK:", sorted(column_sets.keys()))
if errors:
    print("  Años con error:", sorted(errors.keys()))


Verificando estructura de archivos...
  2014: 20 columnas (encoding=utf-8)
  2015: 20 columnas (encoding=utf-8)
  2016: 20 columnas (encoding=utf-8)
  2017: 20 columnas (encoding=utf-8)
  2018: 20 columnas (encoding=utf-8)
  2019: 20 columnas (encoding=utf-8)
  2020: 19 columnas (encoding=utf-8)
  2021: 19 columnas (encoding=utf-8)
  2022: 19 columnas (encoding=utf-8)
  2023: 19 columnas (encoding=utf-8)
  2024: 19 columnas (encoding=utf-8)

Comparando columnas entre años...

  Diferencias detectadas en 2015 (base=2014):
    - Faltan: {'sending_organisation', 'mobility_duration_-_calendar_days', 'receiving_organisation'}
    - Nuevas: {'mobility_duration', 'sending_organization', 'receiving_organization'}

  Diferencias detectadas en 2016 (base=2014):
    - Faltan: {'sending_organisation', 'mobility_duration_-_calendar_days', 'receiving_organisation'}
    - Nuevas: {'mobility_duration', 'sending_organization', 'receiving_organization'}

  Diferencias detectadas en 2017 (base=2014):
   

In [3]:
# En el bloque anterior comprobé que existen diferencias pequeñas entre años en los nombres de columnas.
# En este bloque ya cargo los CSV completos y aplico una estrategia de unificación para poder concatenarlos:
# - Normalizo nombres de columnas (minúsculas, guiones bajos, etc.)
# - Renombro columnas equivalentes a nombres canónicos (RENAME_MAP)
# - Elimino columnas residuales tipo "Unnamed"
# - Añado source_file_year para trazabilidad
# - Alineo todas las columnas entre años y concateno en df_total


folder_path = "bases_datos_erasmus"
years = range(2014, 2025)

def normalize_columns(cols):
    """
    Normaliza nombres de columnas para que todos los años sigan el mismo formato.
    """
    out = []
    for c in cols:
        c = str(c).strip().lower()
        c = re.sub(r"\s+", " ", c)
        c = c.replace(" ", "_")
        c = c.replace("/", "_")
        c = re.sub(r"[()]", "", c)
        c = re.sub(r"__+", "_", c)
        out.append(c)
    return out

# Mapeo a nombres canónicos (columnas equivalentes entre años)
RENAME_MAP = {
    "sending_organisation": "sending_organization",
    "receiving_organisation": "receiving_organization",
    "mobility_duration_-_calendar_days": "mobility_duration",
    "mobility_duration_in_days": "mobility_duration",
    "mobility_start_year_month": "mobility_start_month",
    "actual_participants_contracted_projects": "actual_participants",
}

def read_csv_with_fallback(path, sep=";"):
    """
    Lee el CSV probando distintos encodings para evitar fallos de lectura.
    Devuelve (df, encoding_usado).
    """
    encodings = ["utf-8", "utf-8-sig", "latin1"]
    last_error = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, sep=sep, low_memory=False, encoding=enc)
            return df, enc
        except Exception as e:
            last_error = e
    raise last_error

df_list = []
all_cols = set()

print("Cargando y unificando archivos por año...")

for year in years:
    file_name = f"Erasmus-KA1-Mobility-Data-{year}.csv"
    file_path = os.path.join(folder_path, file_name)

    try:
        # 1) Cargo el CSV del año
        df, used_enc = read_csv_with_fallback(file_path, sep=";")

        # 2) Normalizo nombres de columnas y aplico renombrado canónico
        df.columns = normalize_columns(df.columns)
        df = df.rename(columns=RENAME_MAP)

        # 3) Elimino columnas residuales tipo "Unnamed"
        unnamed_cols = [c for c in df.columns if c.startswith("unnamed")]
        if unnamed_cols:
            df = df.drop(columns=unnamed_cols)

        # 4) Añado trazabilidad del año de origen
        df["source_file_year"] = year

        # 5) Acumulo DF y columnas para alinear al final
        df_list.append(df)
        all_cols |= set(df.columns)

        print(f" {year} cargado: {df.shape[0]:,} filas, {df.shape[1]} columnas (encoding={used_enc})")

    except Exception as e:
        print(f" Error en {year}: {e}")

if not df_list:
    raise ValueError("No se pudo cargar ningún archivo. Revisa la ruta, nombres y separador.")

# Orden fijo de columnas para que el dataset final sea estable
all_cols = sorted(all_cols)

# Alineo todos los años al mismo conjunto de columnas
df_list_aligned = [d.reindex(columns=all_cols) for d in df_list]

# Concateno todo en df_total
df_total = pd.concat(df_list_aligned, ignore_index=True)

print(f"\n{'='*60}\nARCHIVOS UNIDOS\n{'='*60}")
print(f"Total registros: {df_total.shape[0]:,}")
print(f"Total columnas:  {df_total.shape[1]}")


Cargando y unificando archivos por año...
 2014 cargado: 235,702 filas, 21 columnas (encoding=utf-8)
 2015 cargado: 553,475 filas, 21 columnas (encoding=utf-8)
 2016 cargado: 603,003 filas, 21 columnas (encoding=utf-8)
 2017 cargado: 643,036 filas, 21 columnas (encoding=utf-8)
 2018 cargado: 690,757 filas, 21 columnas (encoding=utf-8)
 2019 cargado: 739,730 filas, 21 columnas (encoding=utf-8)
 2020 cargado: 276,660 filas, 20 columnas (encoding=utf-8)
 2021 cargado: 52,109 filas, 20 columnas (encoding=utf-8)
 2022 cargado: 365,106 filas, 20 columnas (encoding=utf-8)
 2023 cargado: 842,756 filas, 20 columnas (encoding=utf-8)
 2024 cargado: 952,741 filas, 20 columnas (encoding=utf-8)

ARCHIVOS UNIDOS
Total registros: 5,955,075
Total columnas:  21


In [4]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5955075 entries, 0 to 5955074
Data columns (total 21 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   academic_year           object 
 1   activity_mob            object 
 2   actual_participants     float64
 3   education_level         object 
 4   fewer_opportunities     object 
 5   field                   object 
 6   field_of_education      object 
 7   mobility_duration       int64  
 8   mobility_start_month    object 
 9   participant_age         object 
 10  participant_country     object 
 11  participant_gender      object 
 12  participant_profile     object 
 13  project_reference       object 
 14  receiving_city          object 
 15  receiving_country       object 
 16  receiving_organization  object 
 17  sending_city            object 
 18  sending_country         object 
 19  sending_organization    object 
 20  source_file_year        int64  
dtypes: float64(1), int64(2), object

In [5]:
# Antes de normalizar, reviso qué valores distintos aparecen en la columna "academic_year".
# Esto me sirve para detectar formatos diferentes (por ejemplo: "2020-2021" vs "2020-21").
print(df_total["academic_year"].unique())


['2013-2014' '2014-2015' '2015-2016' '2016-2017' '2017-2018' '2018-2019'
 '2019-2020' '2020-2021' '2021-22' '2020-21' '2021-2022' '2022-2023'
 '2022-23' '2023-24' '2024-25' '2023-2024']


In [6]:
# Defino una función para dejar todos los valores de "academic_year" con el mismo formato.
# Objetivo: que siempre quede como "YYYY-YY" (por ejemplo, "2020-21").
def norm_academic_year(x):
    # Si el valor es nulo (NaN), lo mantengo como nulo
    if pd.isna(x):
        return pd.NA

    # Paso a string y quito espacios por si vienen valores tipo " 2020-2021 "
    s = str(x).strip()

    # Caso 1: formato completo "YYYY-YYYY" (ej. "2020-2021")
    # En este caso, lo convierto a "YYYY-YY" (ej. "2020-21")
    m = re.fullmatch(r"(\d{4})-(\d{4})", s)
    if m:
        return f"{m.group(1)}-{m.group(2)[-2:]}"

    # Caso 2: formato ya reducido "YYYY-YY" (ej. "2020-21")
    # Si ya está bien, lo dejo tal cual
    m = re.fullmatch(r"(\d{4})-(\d{2})", s)
    if m:
        return s

    # Si aparece algún formato raro que no encaja con los anteriores,
    # lo devuelvo igual para poder revisarlo más adelante.
    return s


# Aplico la normalización y sobrescribo la columna con los valores ya unificados
df_total["academic_year"] = df_total["academic_year"].apply(norm_academic_year)

# Comprobación rápida: vuelvo a mirar los valores únicos para asegurarme de que quedó homogéneo
print(df_total["academic_year"].unique())


['2013-14' '2014-15' '2015-16' '2016-17' '2017-18' '2018-19' '2019-20'
 '2020-21' '2021-22' '2022-23' '2023-24' '2024-25']


In [7]:
# Este bloque lo uso para revisar duplicados en el dataset final.
# Distingo dos casos:
# 1) Duplicados exactos: filas idénticas en todas las columnas (incluyendo academic_year).
# 2) Filas repetidas ignorando academic_year: mismas características pero en distintos cursos,
#    lo cual podría pasar si un registro se repite entre años o si hay una forma de “solapamiento”.
col_year = "academic_year"

# Columnas que muestro como ejemplo cuando encuentro duplicados
# (solo para inspección rápida y que el output no sea enorme)
cols_show = ["academic_year", "sending_country", "receiving_city", "participant_age", "mobility_duration"]


# Asigno df_total a una variable por comodidad.
# No hago copy() para no gastar memoria extra, ya que aquí no modifico el DataFrame.
df = df_total


# 1) Duplicados exactos (todas las columnas iguales)
n = len(df)
n_dup = df.duplicated().sum()
print(f"Duplicados exactos: {n_dup:,} ({n_dup / n * 100:.4f}%)")

# Si existen, muestro un ejemplo de algunas filas para ver el patrón
if n_dup:
    print("\nEjemplo duplicados exactos (5 filas):")
    print(df.loc[df.duplicated(keep=False), cols_show].head(5))


# 2) Filas repetidas ignorando academic_year
# Primero convierto academic_year a un año numérico (me quedo con el primer año: "2015-16" -> 2015)
# Esto me sirve solo para analizar si un mismo "grupo" aparece en más de un año.
year_temp = pd.to_numeric(df[col_year].astype(str).str[:4], errors="coerce").astype("Int64")

# Creo una lista con todas las columnas excepto academic_year para comparar “todo menos el curso”
cols_compare = [c for c in df.columns if c != col_year]

# Para detectar repetidos de forma eficiente, creo un hash por fila usando esas columnas
h = pd.util.hash_pandas_object(df[cols_compare], index=False)

# Marco como repetidas las filas cuyo hash aparece más de una vez
mask_rep = h.duplicated(keep=False)
print(f"\nFilas repetidas ignorando '{col_year}': {mask_rep.sum():,}")

# Ahora miro cuántos de esos grupos aparecen en más de un año distinto
tmp = pd.DataFrame({"h": h[mask_rep], "y": year_temp[mask_rep]})
multi = tmp.groupby("h")["y"].nunique()

# Cuento cuántos hashes tienen más de un año asociado (multi-año)
n_multi_groups = (multi > 1).sum()
print(f"Grupos repetidos en múltiples años: {n_multi_groups:,}")

# Si existe algún caso multi-año, muestro un ejemplo para entenderlo mejor
if n_multi_groups:
    ex_hash = multi[multi > 1].index[0]
    print("\nEjemplo multi-año (5 filas):")
    print(df.loc[h == ex_hash, cols_show].head(5))


Duplicados exactos: 9,165 (0.1539%)

Ejemplo duplicados exactos (5 filas):
     academic_year sending_country receiving_city participant_age  \
1078       2013-14    AT - Austria        Demonte              15   
1079       2013-14    AT - Austria        Demonte              19   
1080       2013-14    AT - Austria        Demonte              15   
1081       2013-14    AT - Austria        Demonte              19   
1741       2014-15    AT - Austria        TAMPERE              20   

      mobility_duration  
1078                 28  
1079                 28  
1080                 28  
1081                 28  
1741                124  

Filas repetidas ignorando 'academic_year': 18,381
Grupos repetidos en múltiples años: 96

Ejemplo multi-año (5 filas):
        academic_year sending_country receiving_city participant_age  \
5584359       2023-24     PL - Poland        SEVILLA              17   
5584360       2024-25     PL - Poland        SEVILLA              17   

         mobility

In [8]:
# En el análisis anterior vi que había duplicados exactos (filas idénticas en todas las columnas).
# Como son registros repetidos al 100%, decido eliminarlos para no contar dos veces la misma observación.
# Mantengo la primera aparición de cada fila y elimino el resto.

n_before = len(df_total)
n_dup = df_total.duplicated().sum()

print(f"Registros antes: {n_before:,}")
print(f"Duplicados exactos detectados: {n_dup:,} ({n_dup / n_before * 100:.4f}%)")

# Elimino duplicados exactos y reseteo el índice para dejar el DataFrame limpio
df_total = df_total.drop_duplicates(keep="first").reset_index(drop=True)

n_after = len(df_total)
print(f"Registros después: {n_after:,}")
print(f"Eliminados: {n_before - n_after:,}")

# Comprobación final: confirmo que ya no quedan duplicados exactos
print("Duplicados restantes:", df_total.duplicated().sum())


Registros antes: 5,955,075
Duplicados exactos detectados: 9,165 (0.1539%)
Registros después: 5,945,910
Eliminados: 9,165
Duplicados restantes: 0


In [9]:
# Reviso los valores más frecuentes de "mobility_start_month" para entender
# por qué la columna está como tipo object (normalmente pasa por formatos mezclados,
# valores raros, o porque viene como texto "YYYY-MM").
df_total["mobility_start_month"].value_counts(dropna=False).head(20)


mobility_start_month
2024-09    168684
2019-09    166655
2018-09    158946
2023-09    155014
2017-09    151617
2016-09    145404
2015-09    138285
2014-09    124833
2024-04    108741
2024-02     93815
2024-05     92027
2023-10     87645
2024-03     83071
2024-10     79464
2023-05     78943
2023-02     76198
2020-02     73570
2023-03     72509
2024-06     70840
2019-02     70045
Name: count, dtype: int64

In [10]:
# No modifico la columna original para poder volver atrás si hace falta.
# Creo una columna auxiliar en formato fecha ("mobility_start_ym") a partir del texto.
# Uso errors="coerce" para que los valores que no encajen con el formato pasen a NaT (nulo de fecha).
df_total["mobility_start_ym"] = pd.to_datetime(
    df_total["mobility_start_month"].astype(str),
    format="%Y-%m",
    errors="coerce"
)

# A partir de la fecha, saco el año y el mes como columnas separadas.
# Uso Int64 (nullable) para permitir nulos sin que se conviertan en float.
df_total["mobility_start_year"] = df_total["mobility_start_ym"].dt.year.astype("Int64")
df_total["mobility_start_month_num"] = df_total["mobility_start_ym"].dt.month.astype("Int64")

# Comprobaciones rápidas:
# 1) Cuántos valores no se pudieron convertir (quedaron como NaT)
print("Nulos mobility_start_ym:", df_total["mobility_start_ym"].isna().sum())

# 2) Distribución de meses (incluyendo nulos) para ver si tiene sentido
print(df_total["mobility_start_month_num"].value_counts(dropna=False).sort_index())


Nulos mobility_start_ym: 0
mobility_start_month_num
1      419047
2      594997
3      478565
4      471914
5      457854
6      374451
7      387487
8      486931
9     1330370
10     559639
11     281515
12     103140
Name: count, dtype: Int64


In [11]:
# 1) Comprobación rápida: cuántos nulos hay y si existen valores con decimales
# En teoría "actual_participants" debería ser un número entero y > 0.
print("NaN actual_participants:", df_total["actual_participants"].isna().sum())
print("Decimales raros:", (df_total["actual_participants"].dropna() % 1 != 0).sum())


NaN actual_participants: 0
Decimales raros: 1


In [12]:
# 2) Inspección: si hay decimales, localizo las filas para ver de dónde vienen
# (me interesa especialmente el año de origen del archivo y el academic_year)
mask_dec = (df_total["actual_participants"] % 1 != 0)
df_total.loc[mask_dec, ["actual_participants", "source_file_year", "academic_year"]].head(10)


,actual_participants,source_file_year,academic_year
4816106,4.7,2023,2023-24


In [13]:
# 3) Limpieza: convierto a nulo los valores imposibles y dejo la columna como entero nullable
# - Si tiene decimales, lo considero inválido (no se puede tener 4.7 participantes)
# - Si es 0, también lo considero inválido (en este dataset no tiene sentido 0 participantes)
col = "actual_participants"

mask_nonint = df_total[col].notna() & ((df_total[col] % 1) != 0)
mask_zero = (df_total[col] == 0)

df_total.loc[mask_nonint | mask_zero, col] = pd.NA
df_total[col] = df_total[col].astype("Int64")


In [14]:
# Antes de limpiar la edad, hago una revisión rápida de valores extremos.
# La idea es convertir "participant_age" a numérico en una variable auxiliar (age_num)
# para poder detectar edades imposibles (<= 0) o demasiado altas (>= 65).

# 1) Convertir a numérico
age_num = pd.to_numeric(
    df_total["participant_age"].replace({"-": pd.NA, "": pd.NA, "0": 0}),
    errors="coerce"
)

# 2) Revisar valores <= 0 (edades negativas o cero no tienen sentido)
print("<= 0 (conteo):", (age_num <= 0).sum())
print(df_total.loc[age_num <= 0, ["participant_age", "academic_year", "source_file_year"]].head(20))

# 3) Revisar valores >= 65 (en este análisis considero que son valores extremos / poco realistas)
print("\n>= 65 (conteo):", (age_num >= 65).sum())
print(df_total.loc[age_num >= 65, ["participant_age", "academic_year", "source_file_year"]].head(20))

# 4) Distribución rápida de esos extremos para ver qué valores aparecen con más frecuencia
print("\nValue counts (<=0):")
print(df_total.loc[age_num <= 0, "participant_age"].value_counts(dropna=False).head(20))

print("\nValue counts (>=65):")
print(df_total.loc[age_num >= 65, "participant_age"].value_counts(dropna=False).head(20))


<= 0 (conteo): 82785
     participant_age academic_year  source_file_year
61                -1       2013-14              2014
126               -2       2013-14              2014
162               -1       2013-14              2014
175               -2       2013-14              2014
183               -1       2014-15              2014
237               -2       2013-14              2014
560               -1       2013-14              2014
663               -1       2013-14              2014
821                0       2014-15              2014
942               -1       2013-14              2014
1011              -1       2013-14              2014
1147               0       2013-14              2014
1343              -1       2013-14              2014
1602              -1       2013-14              2014
1606               0       2013-14              2014
1642              -1       2013-14              2014
5851              -1       2013-14              2014
5910              -1     

In [15]:
# Ahora sí hago la limpieza final de "participant_age".
# 1) Convierto a numérico para que lo que no sea un número pase a NaN.
# 2) Marco como nulos los valores fuera del rango que considero razonable para el análisis.
# 3) Guardo el resultado como entero nullable (Int64), para permitir nulos sin convertir a float.

age = pd.to_numeric(
    df_total["participant_age"].replace({"-": pd.NA, "": pd.NA}),
    errors="coerce"
)

# Rango que considero válido: 10 a 65 años
# Lo de fuera se deja como NA para no inventar datos
age = age.mask((age < 10) | (age > 65))

df_total["participant_age"] = age.astype("Int64")

# Comprobación rápida: número de nulos y edades más frecuentes
print("NaN en participant_age:", df_total["participant_age"].isna().sum())
print(df_total["participant_age"].value_counts(dropna=False).head(15))


NaN en participant_age: 621084
participant_age
21      680917
<NA>    621084
20      590851
22      569740
23      472454
24      348161
19      314029
18      262116
25      237262
17      217568
26      161223
16      123771
27      116001
28       88994
29       72125
Name: count, dtype: Int64


In [16]:
# Limpieza básica de "mobility_duration" (duración de la movilidad en días).
# Aquí me centro en un error claro: duraciones = 0, que no tienen sentido para una movilidad real.
# Primero cuantifico cuántos ceros hay, luego los convierto a nulo (NA) y saco un resumen rápido.

col = "mobility_duration"

# 1) Conteo de ceros (valores imposibles o errores de registro)
n0 = (df_total[col] == 0).sum()
print(f"Duración = 0 días: {n0:,} ({(n0 / len(df_total)) * 100:.4f}%) -> se convierte a NA")

# 2) Reemplazo 0 por NA
df_total.loc[df_total[col] == 0, col] = pd.NA

# 3) Resumen rápido tras la limpieza
print("Nulos tras limpieza:", df_total[col].isna().sum())
print("Min/Mediana/Max:", df_total[col].min(), df_total[col].median(), df_total[col].max())


Duración = 0 días: 16,069 (0.2703%) -> se convierte a NA
Nulos tras limpieza: 16069
Min/Mediana/Max: 1.0 29.0 1095.0


In [17]:
# Revisión adicional: duraciones muy largas.
# No las elimino automáticamente, pero las cuantifico para saber si hay valores extremos
# (por ejemplo, movilidades de más de un año o más de dos años).

col = "mobility_duration"

print(">365 días:", (df_total[col] > 365).sum())
print(">730 días:", (df_total[col] > 730).sum())


>365 días: 3190
>730 días: 81


In [18]:
# En varias columnas de texto hay valores que en realidad significan "desconocido" o "no aplica",
# pero vienen escritos como strings (por ejemplo "unknown", "n/a", "???", "-").
# Para poder trabajar mejor con los datos, unifico todos esos casos y los convierto a NA.
# Así, los análisis de nulos y las limpiezas posteriores son más consistentes.

# Tokens típicos de "nulo encubierto" (no incluyo "0" por defecto porque en algunas columnas puede ser un valor válido)
missing_tokens = {
    "unknown", "not specified", "none", "n/a", "na",
    "?", "??", "???", "????",
    "-", "--", "---",
    "_", "__",
    "? unknown ?", "??? - ? unknown ?"
}

# Selecciono columnas de tipo texto (object o category)
text_cols = df_total.select_dtypes(include=["object", "category"]).columns

# Reemplazo general en columnas de texto:
# - quito espacios
# - paso a minúsculas para comparar con los tokens
# - convierto a NA cuando es cadena vacía o coincide con un token de missing
for c in text_cols:
    s = df_total[c].astype("string").str.strip()
    s_low = s.str.lower()
    df_total[c] = s.mask((s == "") | (s_low.isin(missing_tokens)), pd.NA)

# Caso específico: en "receiving_city" aparece el string "0" como valor inválido en algunos registros.
# En este dataset lo trato como nulo para no confundirlo con una ciudad real.
if "receiving_city" in df_total.columns:
    df_total.loc[
        df_total["receiving_city"].astype("string").str.strip() == "0",
        "receiving_city"
    ] = pd.NA

# Resumen rápido: muestro las columnas con más valores nulos para tener una visión general
na_counts = df_total.isna().sum().sort_values(ascending=False)
print("Top 10 columnas con más NA:")
print(na_counts.head(10))

Top 10 columnas con más NA:
project_reference         2481891
education_level           2179182
field_of_education        1993699
participant_age            621084
receiving_organization     567585
sending_organization       323004
fewer_opportunities        229867
participant_country         35709
participant_gender          33335
mobility_duration           16069
dtype: int64


In [19]:
# En "participant_profile" encuentro valores en distintos formatos (por ejemplo en mayúsculas).
# Para evitar que "LEARNERS" y "Learner" se traten como categorías distintas, unifico etiquetas.
# Después convierto la columna a tipo category para que quede más ordenada y ocupe menos memoria.

df_total["participant_profile"] = df_total["participant_profile"].replace({
    "LEARNERS": "Learner",
    "STAFF": "Staff"
})

df_total["participant_profile"] = df_total["participant_profile"].astype("category")

# Comprobación rápida: veo cuántos registros hay de cada perfil (incluyendo nulos)
print(df_total["participant_profile"].value_counts(dropna=False))


participant_profile
Learner    4428605
Staff      1500857
<NA>          8841
Other         7607
Name: count, dtype: int64


In [20]:
# La columna "fewer_opportunities" viene codificada como 0/1 (a veces como texto y a veces como número).
# Para que sea más fácil de interpretar, la convierto a etiquetas "No" y "Yes".
# Después la paso a tipo category, ya que solo tiene unas pocas categorías posibles.

df_total["fewer_opportunities"] = df_total["fewer_opportunities"].replace({
    "0": "No", 0: "No",
    "1": "Yes", 1: "Yes"
}).astype("category")

# Comprobación rápida: distribución de valores (incluyendo nulos)
print(df_total["fewer_opportunities"].value_counts(dropna=False))


fewer_opportunities
No      5001227
Yes      714816
<NA>     229867
Name: count, dtype: int64


In [21]:
# A partir de "education_level" quiero extraer el nivel ISCED (si aparece en el texto).
# En el dataset suele venir en formatos como:
# "ISCED-6 - First cycle / Bachelor’s ..." o similares.
# La idea es quedarme solo con el número (1..9) para poder agrupar y filtrar mejor.

# 1) Extraer el número de ISCED (si existe)
df_total["isced_level"] = (
    df_total["education_level"]
      .astype("string")                          # aseguro formato string para trabajar con .str
      .str.extract(r"ISCED-(\d)", expand=False)  # capturo el dígito después de "ISCED-"
      .astype("Int64")                           # lo dejo como entero nullable (permite NA)
)

# 2) Crear un grupo simplificado (más útil para el análisis del proyecto)
# - HE (6-8): educación superior (Bachelor/Master/Doctorate)
# - Pre-tertiary (1-5): niveles previos
# - ISCED-9 / Other: otros casos
# Si no se puede extraer ISCED, se queda como NA.
df_total["isced_group"] = pd.Series(pd.NA, index=df_total.index, dtype="string")

df_total.loc[df_total["isced_level"].between(6, 8), "isced_group"] = "HE (6-8)"
df_total.loc[df_total["isced_level"].between(1, 5), "isced_group"] = "Pre-tertiary (1-5)"
df_total.loc[df_total["isced_level"] == 9, "isced_group"] = "ISCED-9 / Other"

df_total["isced_group"] = df_total["isced_group"].astype("category")

# Comprobación rápida: distribución de niveles ISCED y de los grupos creados
print(df_total["isced_level"].value_counts(dropna=False).sort_index())
print(df_total["isced_group"].value_counts(dropna=False))


isced_level
1          3996
2         55627
3        405814
4        122182
5        108643
6       1908854
7       1007928
8        104122
9         49562
<NA>    2179182
Name: count, dtype: Int64
isced_group
HE (6-8)              3020904
<NA>                  2179182
Pre-tertiary (1-5)     696262
ISCED-9 / Other         49562
Name: count, dtype: int64


In [22]:
# La columna "activity_mob" mezcla texto largo con un código al principio (en muchos casos).
# Para analizarla mejor, separo:
# 1) Un "activity_code" (si existe) extraído del inicio del texto.
# 2) Un "activity_group" con categorías más generales (HE, VET, Youth, etc.).
# Esto me permite filtrar y resumir sin depender de cientos de etiquetas distintas.

# Limpio espacios para evitar problemas al extraer el código
s = df_total["activity_mob"].astype("string").str.strip()

# 1) Extraigo el código del formato "CODIGO - descripción"
# Ejemplos típicos: "HE-SMS - ...", "LM-VET - ...", etc.
code = s.str.extract(r"^([A-Z]{1,4}(?:-[A-Z0-9]{1,10})+)\s*-\s*", expand=False)
df_total["activity_code"] = code.astype("category")


def group_from_activity(text, code):
    """
    Asigna una categoría general a cada registro de activity_mob.
    Primero intento clasificar por el código (si está disponible).
    Si no hay código o el formato es antiguo, intento clasificar por palabras clave del texto.
    """
    if pd.isna(text):
        return pd.NA

    t = str(text).lower()
    c = str(code) if pd.notna(code) else ""

    # 1) Clasificación por código (cuando existe)
    if c:
        if c.startswith("HE-"):
            return "HE"
        if c.startswith("LM-") and "VET" in c:
            return "VET"
        if c.startswith("LM-") and ("EXCH" in c or "YOU" in c):
            return "Youth/Volunteering"
        if c.startswith("LM-") and "PUPIL" in c:
            return "School"
        if c.startswith("LM-") and "ADULT" in c:
            return "Adult"
        if c.startswith("SM-") or c.startswith("OA-"):
            return "Staff/Training"

    # 2) Clasificación por texto (para casos antiguos o sin código)
    if (
        "student mobility for studies" in t
        or "student mobility for traineeships" in t
        or "higher education" in t
    ):
        return "HE"

    if (
        "youth exchanges" in t
        or "mobility of youth workers" in t
        or "european voluntary service" in t
        or "volunteering" in t
    ):
        return "Youth/Volunteering"

    if (
        "vet learners" in t
        or "mobility of vet learners" in t
        or "erasmuspro" in t
        or "vet" in t
    ):
        return "VET"

    if (
        "staff mobility" in t
        or "staff training" in t
        or "structured courses/training events" in t
        or "training/teaching assignments abroad" in t
        or "teaching/training assignments abroad" in t
        or "job shadowing" in t
    ):
        return "Staff/Training"

    if "school pupils" in t or "pupil" in t:
        return "School"

    if "adult learners" in t:
        return "Adult"

    # Visitas/preparación/hosting (incluye Advance Planning Visit)
    if (
        "advance planning visit" in t
        or "preparatory visit" in t
        or "invited experts" in t
        or "hosting teachers" in t
    ):
        return "Staff/Training"

    # Si no encaja con nada, lo dejo como "Other"
    return "Other"


# Aplico la función para crear el grupo final.
# Uso zip para pasar a la vez el texto y el código extraído.
df_total["activity_group"] = [
    group_from_activity(txt, c)
    for txt, c in zip(df_total["activity_mob"], df_total["activity_code"])
]

df_total["activity_group"] = df_total["activity_group"].astype("category")

# Comprobación rápida: distribución de grupos
print(df_total["activity_group"].value_counts(dropna=False))


activity_group
HE                    2932465
Staff/Training        1150150
Youth/Volunteering     984875
VET                    821099
School                  51514
Adult                    3468
NaN                      2339
Name: count, dtype: int64


In [23]:
# Este bloque sirve para agrupar "field_of_education" en categorías amplias (ISCED-F broad fields).
# En el dataset, esta columna puede venir con:
# - un código al inicio (por ejemplo "0410 - Business and administration")
# - o directamente texto (a veces sin código)
# Lo que hago es:
# 1) Intentar extraer un código si existe al principio (4 dígitos / 2 dígitos / 1 dígito)
# 2) Si no hay código, aplicar reglas por palabras clave para asignar un macro-campo
# 3) Si no encaja con nada, lo marco como "99 - Not classified"
#
# Nota: muchos registros pueden venir con NA en field_of_education, por eso aparece un número alto de NaN.

COL = "field_of_education"  # ajusta si tu columna tiene espacios

# Etiquetas amplias (2 dígitos) para ISCED broad fields
broad_labels = {
    "00": "00 - Generic programmes and qualifications",
    "01": "01 - Education",
    "02": "02 - Arts and humanities",
    "03": "03 - Social sciences, journalism and information",
    "04": "04 - Business, administration and law",
    "05": "05 - Natural sciences, mathematics and statistics",
    "06": "06 - Information and Communication Technologies",
    "07": "07 - Engineering, manufacturing and construction",
    "08": "08 - Agriculture, forestry, fisheries and veterinary",
    "09": "09 - Health and welfare",
    "10": "10 - Services",
    "99": "99 - Not classified",
}

# Reglas por texto: si la descripción contiene ciertas palabras clave, asigno un macro-código
rules = [
    (r"\beducation\b|teacher training|pre-?school", "01"),
    (r"arts?|fine arts|handicrafts|music|performing|audio-visual|fashion|design|humanities|"
     r"history|archaeology|philosophy|ethics|religion|theology|languages?|linguistics|literature", "02"),
    (r"social and behavioural|economics|political|psychology|sociology|cultural studies|"
     r"journalism|reporting|library|archival", "03"),
    (r"accounting|taxation|finance|banking|insurance|management|administration|marketing|advertising|"
     r"secretarial|office work|wholesale|retail|business\b|law\b|legal|juris", "04"),
    (r"biolog|biochem|chemistry|earth sciences|physics|physical sciences|mathematics|statistics|"
     r"natural sciences|environment", "05"),
    (r"\bict\b|information and communication technologies|computer use|database|network|software|applications", "06"),
    (r"engineering|electricity|energy\b|electronics|automation|mechanics|metal trades|motor vehicles|ships|aircraft|"
     r"manufacturing|processing|materials|textiles|mining|extraction|architecture|building|civil engineering|construction", "07"),
    (r"agricult|crop|livestock|horticult|forestry|fisheries|veterinar", "08"),
    (r"\bhealth\b|medicine|medical|nursing|midwifery|therapy|rehabilitation|pharmacy|dental|"
     r"welfare|care of the elderly|disabled adults|child care|youth services|social work|counselling", "09"),
    (r"personal services|domestic services|hair|beauty|hotel|restaurants|catering|"
     r"sports|travel|tourism|leisure|hygiene|occupational health and safety|community sanitation|"
     r"security services|military|defence|transport|\bservices\b", "10"),
    (r"generic programmes|eqf-\d", "00"),
    (r"^not-?classified\b", "99"),
]

def to_macro_code(val):
    # Si el valor es nulo, lo mantengo como NA
    if pd.isna(val):
        return pd.NA

    s = str(val).strip().lower()

    # 1) Si empieza con código, lo intento convertir a un macro-código de 2 dígitos
    # Acepto "#### - ...", "## - ..." o "# - ..."
    m = re.match(r"^(\d{4}|\d{2}|\d)\s*-\s*", s)
    if m:
        raw = m.group(1)
        if len(raw) == 4:
            code2 = raw[:2]
        elif len(raw) == 2:
            code2 = raw
        else:
            code2 = f"0{raw}"

        if code2 in broad_labels:
            return code2

    # 2) Si no hay código, aplico reglas de palabras clave
    for pat, code in rules:
        if re.search(pat, s):
            return code

    # 3) Si no encaja, lo marco como no clasificado
    return "99"


# Aplico la función para generar el macro-código y su etiqueta
df_total["isced_macro_code"] = df_total[COL].apply(to_macro_code)
df_total["isced_macro"] = df_total["isced_macro_code"].map(broad_labels).astype("category")

# Resumen rápido para ver si la clasificación tiene sentido
print(df_total["isced_macro"].value_counts(dropna=False))
print("No clasificados (99):", (df_total["isced_macro_code"] == "99").sum())


isced_macro
NaN                                                     1993699
04 - Business, administration and law                    814678
02 - Arts and humanities                                 708864
07 - Engineering, manufacturing and construction         559492
03 - Social sciences, journalism and information         411092
01 - Education                                           405750
09 - Health and welfare                                  311763
10 - Services                                            266341
05 - Natural sciences, mathematics and statistics        223087
06 - Information and Communication Technologies          137752
08 - Agriculture, forestry, fisheries and veterinary     108669
99 - Not classified                                        4717
00 - Generic programmes and qualifications                    6
Name: count, dtype: int64
No clasificados (99): 4717


In [24]:
# En el dataset hay tres columnas relacionadas con países:
# - participant_country: suele venir solo como nombre (sin código delante).
# - sending_country / receiving_country: suelen venir como "XX - Country name" (XX = ISO2).
#
# Problema:
# - El mismo país puede aparecer con nombres distintos (por ejemplo "Türkiye" vs "Turkey"),
#   o con variantes largas ("Iran (Islamic Republic of)").
# - En sending/receiving, para un mismo código ISO2 puede haber pequeñas diferencias en el nombre
#   (espacios, capitalización, variantes), lo que rompe conteos si no se unifica.
#
# Estrategia:
# 1) participant_country: limpio espacios y aplico un diccionario de equivalencias (name_map).
# 2) sending_country y receiving_country: normalizo por código ISO2:
#    - extraigo el código y el nombre,
#    - elijo el nombre más frecuente como “canónico” por código,
#    - creo columnas auxiliares <col>_code y <col>_name,
#    - reconstruyo la columna original en formato "XX - NombreCanónico".
# 3) Estándar global: fuerzo que sending y receiving usen el mismo nombre canónico por código,
#    para que un código no tenga un nombre distinto según la columna.
# 4) Check final: verifico que no quedan códigos con más de un nombre.



# 1) Normalización de participant_country (solo nombre, sin ISO2)
name_map = {
    "Türkiye": "Turkey",
    "Czechia": "Czech Republic",
    "Russian Federation": "Russia",
    "Moldova (Republic of)": "Moldova",
    "Republic of Moldova": "Moldova",
    "China (People's Republic of)": "China",
    "Iran (Islamic Republic of)": "Iran",
    "Syrian Arab Republic": "Syria",
    "Viet Nam": "Vietnam",
    "Tanzania (United Republic of)": "Tanzania",
    "Korea (Republic of)": "South Korea",
    "Korea (Democratic People's Republic of)": "North Korea",
    "Cabo Verde": "Cape Verde",
    "Åland islands": "Aland Islands",
    "United States of America": "United States",
    "United States Minor outlying islands": "United States Minor Outlying Islands",
    "St Lucia": "Saint Lucia",
    "St Kitts and Nevis": "Saint Kitts and Nevis",
    "St Vincent and the Grenadines": "Saint Vincent and the Grenadines",
    "Isle Of Man": "Isle of Man",
    "The Republic of North Macedonia": "North Macedonia",
    "Kosovo * UN resolution": "Kosovo",
    "Congo (Democratic Republic of)": "Democratic Republic of the Congo",
    "Lao (People's Democratic Republic)": "Laos",
    "Lao People's Democratic Republic": "Laos",
    "Brunei Darussalam": "Brunei",
    "French Southern and Antarctic Territories": "French Southern Territories",
}

df_total["participant_country"] = (
    df_total["participant_country"]
      .astype("string")
      .str.strip()
      .str.replace(r"\s+", " ", regex=True)
      .replace(name_map)
      .astype("category")
)

print("participant_country únicos:", df_total["participant_country"].nunique(dropna=True))


# 2) Normalización por ISO2 para sending_country y receiving_country
def normalize_country_by_code(df, col):
    """
    Normaliza una columna que viene como 'XX - Name' (XX = ISO2).
    Para cada código, se usa el nombre más frecuente como nombre canónico.
    """
    s = df[col].astype("string").str.strip()

    # Validación rápida: compruebo que existe el patrón "XX - ..."
    has_code = s.str.match(r"^[A-Z]{2}\s*[-–]\s*", na=False).any()
    if not has_code:
        raise ValueError(f"{col} no parece venir como 'XX - Name'. No puedo extraer ISO2.")

    # Extraigo código y nombre
    code = s.str.extract(r"^([A-Z]{2})\s*[-–]\s*", expand=False)
    name = s.str.replace(r"^[A-Z]{2}\s*[-–]\s*", "", regex=True).str.strip()
    name = name.str.replace(r"\s+", " ", regex=True)

    # Nombre canónico por código: el más frecuente
    canon_name = (
        pd.DataFrame({"code": code, "name": name})
          .dropna()
          .groupby("code")["name"]
          .agg(lambda x: x.value_counts().idxmax())
    )

    # Guardo columnas auxiliares y reconstruyo la columna original normalizada
    df[col + "_code"] = code.astype("category")
    df[col + "_name"] = code.map(canon_name).astype("category")
    df[col] = (
        df[col + "_code"].astype("string") + " - " + df[col + "_name"].astype("string")
    ).astype("category")

    return df


df_total = normalize_country_by_code(df_total, "sending_country")
df_total = normalize_country_by_code(df_total, "receiving_country")


# 3) Estándar global: mismo nombre por código en sending y receiving
both = pd.concat([
    df_total[["sending_country_code", "sending_country_name"]]
      .rename(columns={"sending_country_code": "code", "sending_country_name": "name"}),
    df_total[["receiving_country_code", "receiving_country_name"]]
      .rename(columns={"receiving_country_code": "code", "receiving_country_name": "name"}),
], ignore_index=True).dropna()

global_name = both.groupby("code")["name"].agg(lambda x: x.value_counts().idxmax())

df_total["sending_country_name"] = df_total["sending_country_code"].map(global_name).astype("category")
df_total["receiving_country_name"] = df_total["receiving_country_code"].map(global_name).astype("category")

df_total["sending_country"] = (
    df_total["sending_country_code"].astype("string") + " - " + df_total["sending_country_name"].astype("string")
).astype("category")

df_total["receiving_country"] = (
    df_total["receiving_country_code"].astype("string") + " - " + df_total["receiving_country_name"].astype("string")
).astype("category")


# 4) Check final: confirmo que no hay códigos con más de un nombre canónico
for base in ["sending_country", "receiving_country"]:
    tmp = df_total[[base + "_code", base + "_name"]].dropna()
    multi = tmp.groupby(base + "_code")[base + "_name"].nunique()
    print("FINAL", base, "códigos con >1 nombre:", int((multi > 1).sum()))


participant_country únicos: 255


C:\Users\alizd\AppData\Local\Temp\ipykernel_4364\3878975602.py:132: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  multi = tmp.groupby(base + "_code")[base + "_name"].nunique()
C:\Users\alizd\AppData\Local\Temp\ipykernel_4364\3878975602.py:132: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  multi = tmp.groupby(base + "_code")[base + "_name"].nunique()


FINAL sending_country códigos con >1 nombre: 0
FINAL receiving_country códigos con >1 nombre: 0


In [25]:
# - Reduzco variaciones de escritura en nombres de ciudades (espacios extra, sufijos tipo CEDEX,
#   puntuación, distritos como "Paris 16", tildes, etc.).
# - Unifico algunas variantes/exónimos frecuentes a una forma estándar en inglés.
# Nota sobre codificación:
# - En algunos registros puede aparecer el carácter "�". Esto suele indicar un problema
#   de encoding ya presente en los datos originales. Aquí no se corrige pero se cuantifica como chequeo de calidad.

def strip_accents(s: str) -> str:
    """
    Quita tildes/diacríticos para reducir variantes del mismo nombre.
    Ejemplo: "Málaga" -> "Malaga".
    """
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    return s.replace("ß", "ss")


# Mapa de exónimos/variantes frecuentes a una forma estándar en inglés
english_map = {
    "Wien": "Vienna",
    "Praha": "Prague",
    "Bruxelles": "Brussels",
    "Roma": "Rome",
    "Milano": "Milan",
    "Lisboa": "Lisbon",
    "Warszawa": "Warsaw",
    "Oporto": "Porto",
    "Bologne": "Bologna",
    "Bolonia": "Bologna",
    "Firenze": "Florence",
    "Torino": "Turin",
    "Kobenhavn": "Copenhagen",
    "Sevilla": "Seville",
}


def clean_city_series(series: pd.Series, english_map: dict) -> pd.Series:
    """
    Limpieza básica de nombres de ciudades.
    Aplico reglas simples para homogenizar el texto sin normalizar ciudad por ciudad.
    """
    s = series.astype("string").str.strip()

    # 1) Normalizar espacios
    s = s.str.replace(r"\s+", " ", regex=True)

    # 2) Quitar sufijos tipo "CEDEX" (comunes en direcciones de Francia)
    s = s.str.replace(r"\s+cedex\s*\d*\s*$", "", regex=True, case=False)

    # 3) Quitar puntuación final típica
    s = s.str.replace(r"[.,;:?]+$", "", regex=True)

    # 4) Quitar números de distrito al final (Paris 16, Dublin 2, etc.)
    s = s.str.replace(r"\s+\d{1,3}\s*$", "", regex=True)

    # 5) Quitar tildes/diacríticos y estandarizar capitalización
    s = s.apply(lambda x: strip_accents(str(x)) if pd.notna(x) else pd.NA)
    s = s.str.title()

    # 6) Unificar exónimos/variantes frecuentes
    s = s.replace(english_map)

    return s


# Aplico la limpieza a ambas columnas de ciudad si existen en el DataFrame
city_cols = ["receiving_city", "sending_city"]

for col in city_cols:
    if col in df_total.columns:
        df_total[col] = clean_city_series(df_total[col], english_map).astype("category")

        # Chequeo ligero: número de únicos y presencia de mojibake (�)
        print(f"\n{col}")
        print("Únicos:", df_total[col].nunique(dropna=True))

        bad = df_total[col].astype("string").str.contains("�", na=False).sum()
        print("Filas con mojibake (�):", bad)

        # Ejemplos mínimos (solo si existe mojibake) para poder inspeccionarlo sin generar mucho output
        if bad > 0:
            print("Ejemplos con mojibake:")
            print(df_total.loc[df_total[col].astype("string").str.contains("�", na=False), col].head(5))



receiving_city
Únicos: 94344
Filas con mojibake (�): 84140
Ejemplos con mojibake:
24           Cefal�
69     St. Julian�S
70     St. Julian�S
71     St. Julian�S
90    San Sebasti�N
Name: receiving_city, dtype: category
Categories (94344, object): ['', ' Berne', ' Pons', ' Puigdalber Barcelona', ..., '�������', '������� ����', '��������', '���������']

sending_city
Únicos: 46021
Filas con mojibake (�): 89959
Ejemplos con mojibake:
12        G�Nserndorf
16    Gro�-Enzersdorf
17    Gro�-Enzersdorf
20         St. P�Lten
21         St. P�Lten
Name: sending_city, dtype: category
Categories (46021, object): ['', ' Paris', '"59155 Faches Thumesnil"', '"59390 Lys Lez Lannoy"', ..., '���������', '���������(Botevgrad)', '����������', '�����������']


In [26]:
# Voy a preparar un dataset filtrado para el análisis (el que luego exportaré para Power BI).
# Pasos:
# 1) Creo year_start a partir de academic_year (por ejemplo "2019-20" -> 2019).
# 2) Filtro el periodo 2017–2024 por año de inicio.
# 3) Comparo criterios para definir HE (ISCED, activity_group, field) como evidencia.
# 4) Defino el dataset final HE por ISCED 6–8 y añado una bandera he_strict.

df = df_total.copy()

df["year_start"] = pd.to_numeric(
    df["academic_year"].astype("string").str.extract(r"(\d{4})")[0],
    errors="coerce"
).astype("Int64")

df_17_24 = df[df["year_start"].between(2017, 2024)].copy()

print(df_17_24["academic_year"].value_counts().sort_index().tail(10))
print("Filas:", len(df_17_24))


academic_year
2017-18    673550
2018-19    727899
2019-20    480060
2020-21     93710
2021-22    351358
2022-23    580488
2023-24    950429
2024-25    323383
Name: count, dtype: Int64
Filas: 4180877


In [27]:
#Comparación de criterios para definir HE (evidencia)
mask_isced = df_17_24["isced_level"].isin([6, 7, 8])
mask_act   = df_17_24["activity_group"].eq("HE")
mask_field = df_17_24["field"].eq("Higher Education")

summary = pd.DataFrame({
    "Filtro": [
        "ISCED 6-8 (criterio base)",
        "activity_group == HE",
        "field == Higher Education",
        "ISCED 6-8 AND activity_group HE (HE estricto)",
        "ISCED 6-8 AND field HE",
        "field HE AND activity_group HE",
        "ISCED 6-8 AND field HE AND activity_group HE",
    ],
    "Filas": [
        mask_isced.sum(),
        mask_act.sum(),
        mask_field.sum(),
        (mask_isced & mask_act).sum(),
        (mask_isced & mask_field).sum(),
        (mask_field & mask_act).sum(),
        (mask_isced & mask_field & mask_act).sum(),
    ]
})

summary["% sobre df_17_24"] = (summary["Filas"] / len(df_17_24) * 100).round(2)
print(summary)


                                          Filtro    Filas  % sobre df_17_24
0                      ISCED 6-8 (criterio base)  2082071             49.80
1                           activity_group == HE  2058366             49.23
2                      field == Higher Education  2312575             55.31
3  ISCED 6-8 AND activity_group HE (HE estricto)  1887691             45.15
4                         ISCED 6-8 AND field HE  2014128             48.17
5                 field HE AND activity_group HE  2058366             49.23
6   ISCED 6-8 AND field HE AND activity_group HE  1887691             45.15


In [28]:
#Dataset final HE para Power BI (HE por ISCED 6–8 + bandera he_strict)
df_17_24["he_isced"] = mask_isced
df_17_24["he_strict"] = mask_isced & (df_17_24["activity_group"] == "HE")

df_he = df_17_24[df_17_24["he_isced"]].copy()

print("HE (ISCED 6-8):", len(df_he))
print("HE estricto (dentro de HE):", df_he["he_strict"].sum())
print(df_he["academic_year"].value_counts().sort_index().tail(10))


HE (ISCED 6-8): 2082071
HE estricto (dentro de HE): 1887691
academic_year
2017-18    358415
2018-19    373111
2019-20    308735
2020-21     81643
2021-22    135559
2022-23    221953
2023-24    396792
2024-25    205863
Name: count, dtype: Int64


In [29]:
df_he.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2082071 entries, 1432186 to 5944929
Data columns (total 37 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   academic_year             string        
 1   activity_mob              string        
 2   actual_participants       Int64         
 3   education_level           string        
 4   fewer_opportunities       category      
 5   field                     string        
 6   field_of_education        string        
 7   mobility_duration         float64       
 8   mobility_start_month      string        
 9   participant_age           Int64         
 10  participant_country       category      
 11  participant_gender        string        
 12  participant_profile       category      
 13  project_reference         string        
 14  receiving_city            category      
 15  receiving_country         category      
 16  receiving_organization    string        
 17  sending

In [30]:
# Dataset final para Power BI: me quedo solo con las columnas que voy a usar en el análisis.
# Para países, conservo ISO2 (code) + nombre (name) porque es más robusto para mapas y para un modelo estrella.

cols_keep = [
    "academic_year", "year_start",
    "mobility_start_ym", "mobility_start_year", "mobility_start_month_num",

    # Países (ISO2 + nombre)
    "sending_country_code", "sending_country_name",
    "receiving_country_code", "receiving_country_name",

    # participant_country viene como nombre (no ISO2 en este pipeline)
    "participant_country",

    # Variables demográficas / flags
    "participant_gender", "participant_profile", "fewer_opportunities",

    # Métricas numéricas
    "participant_age", "mobility_duration", "actual_participants",

    # Educación y actividad
    "isced_level", "isced_group",
    "activity_group",
    "isced_macro",

    # Flag para análisis conservador dentro de HE
    "he_strict",
]

df_he_pbi = df_he[cols_keep].copy()


In [ ]:
# Exporto el dataset final en dos formatos:
# - Parquet (recomendado): comprimido, rápido de cargar, conserva tipos de datos.
# - CSV (universal): para compatibilidad con herramientas que no soporten Parquet.
# La carpeta de salida se crea si no existe.

import os

output_dir = "data/processed"
os.makedirs(output_dir, exist_ok=True)

# 1) Exporto a Parquet (preferido para Power BI)
parquet_path = os.path.join(output_dir, "erasmus_he_2017_2024.parquet")
df_he_pbi.to_parquet(parquet_path, index=False)

# 2) Exporto a CSV (fallback)
csv_path = os.path.join(output_dir, "erasmus_he_2017_2024.csv")
df_he_pbi.to_csv(csv_path, index=False)

# Comprobación rápida: confirmo la exportación
print(f"Exportados {len(df_he_pbi):,} registros en {output_dir}/")
print(f"  - erasmus_he_2017_2024.parquet ({os.path.getsize(parquet_path) / 1e6:.1f} MB)")
print(f"  - erasmus_he_2017_2024.csv ({os.path.getsize(csv_path) / 1e6:.1f} MB)")